In [ ]:
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px  # for data visualization
import plotly.graph_objects as go  # for data visualization
from prophet import Prophet  # for time series forecasting

In [ ]:
data = pd.read_csv("data/data.csv")  # Read the original data from the csv file
data["Months"] = pd.to_datetime(
    data["Months"]
)  # Convert the Months column to datetime format

# remove first two characters from the column Unit_ID
data["Unit_ID"] = data["Unit_ID"].map(lambda x: int(str(x)[2:]))

In [ ]:
data = (
    data.groupby(["Department_ID", "Unit_ID", "Months"])["Expenses"].sum().reset_index()
)  # Group by Department_ID and Months and sum the Expenses so that we get the total expenses for each department in each month

In [ ]:
pivoted_df = data.pivot(
    index="Months", columns=["Department_ID", "Unit_ID"], values="Expenses"
).reset_index()  # Pivot the data so that we have the months as the index and the Department_ID as the columns

In [ ]:
# for d_id in data["Department_ID"].unique():
#     months = pivoted_df["Months"]
#     dep_data = pd.concat([months, pivoted_df[d_id]], axis=1)
#     fig = px.line(
#         dep_data,
#         x="Months",
#         y=dep_data.columns,
#         hover_data={"Months": "|%B %d, %Y"},
#         title="Expenses by Units in Department " + str(d_id),
#     )
#     fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
#     fig.update_layout(
#             xaxis_title="Months",
#             yaxis_title="Expenses",
#             legend_title="Unit ID",
#         )
#     # Save the plot as an image
#     fig.write_image("images/department_" + str(d_id) + ".png")

In [ ]:
department_id = 3100  # Department ID for which we want to forecast the expenses

months = pivoted_df["Months"]
department_data = pd.concat([months, pivoted_df[department_id]], axis=1)

fig = px.line(
    department_data,
    x="Months",
    y=department_data.columns,
    hover_data={"Months": "|%B %d, %Y"},
    title="Expenses by Units in Department " + str(department_id),
)
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.update_layout(
    xaxis_title="Months",
    yaxis_title="Expenses",
    legend_title="Unit ID",
)
fig.show()

In [ ]:
unit_id = 68401  # Unit ID for which we want to forecast the expenses
unit_data = department_data[["Months", unit_id]]

fig = px.line(unit_data, x="Months", y=unit_id)
fig.update_xaxes(dtick="M1", tickformat="%b\n%Y")
fig.update_layout(
    title="Expenses of Unit ID "
    + str(unit_id)
    + " in Department ID "
    + str(department_id),
    xaxis_title="Months",
    yaxis_title="Expenses",
)
fig.show()

In [ ]:
train_data = unit_data.rename(
    columns={"Months": "ds", unit_id: "y"}
)  # Rename the columns to ds and y as required by Prophet

In [ ]:
model = Prophet()  # Initialize Prophet model
model.fit(train_data)  # Fit the model
future = model.make_future_dataframe(
    periods=12, freq="M"
)  # Extend 12 months into the future
forecast = model.predict(future)  # Make predictions

In [ ]:
future["y"] = train_data["y"]  # Add the actual data to the future dataframe
future["yhat"] = forecast["yhat"]  # Add the predicted data to the future dataframe

In [ ]:
fig = px.line(
    future,
    x="ds",
    y="y",
    title="Forecasting for expenses of Unit ID "
    + str(unit_id)
    + " in Department ID "
    + str(department_id),
    labels={"y": "Expenses", "ds": "Months"},
)

fig.add_trace(
    go.Line(x=future["ds"], y=future["yhat"], mode="lines", name="Predicted Expenses")
)
fig.add_scatter(
    x=future["ds"],
    y=future["y"],
    mode="markers",
    marker=dict(color="blue"),
    name="Actual Expenses",
)
fig.add_scatter(
    x=future["ds"],
    y=future["yhat"],
    mode="markers",
    marker=dict(color="red"),
    name="Predicted Expenses",
)
fig.update_layout(xaxis_title="Months", yaxis_title="Expenses", hovermode="closest")

fig.show()